In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/scamps_datatape_450x8x8_train.h5"
valid_tape = "Z:/rppg/scamps_datatape_450x8x8_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)
train, valid = to_tf(train).cache(), to_tf(valid).cache()
#train, valid = to_tf(train).cache(f'{tmp}/ours'), to_tf(valid).cache(f'{tmp}/ours')
#train, valid = to_tf(train), to_tf(valid)

In [14]:
m1 = M_1()
m1.compile(optimizer='adam', loss='mae')
m1.build(input_shape=(None, 450, 8, 8, 3))
print(f'Flops per frame:{get_flops(m1, input_sig=[tf.TensorSpec([1, 450, 8, 8, 3])])/450:.0f}')
m1.summary()

Flops per frame:260557
Model: "m_1_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_12 (Sequential)   (None, 450, 64)           12352     
_________________________________________________________________
spectral_transform_4 (Spectr multiple                  82560     
_________________________________________________________________
conv1d_14 (Conv1D)           multiple                  41024     
_________________________________________________________________
sequential_14 (Sequential)   (None, 450, 64)           256       
_________________________________________________________________
spectral_transform_5 (Spectr multiple                  49792     
_________________________________________________________________
conv1d_16 (Conv1D)           multiple                  10272     
_________________________________________________________________
sequential_16 (Sequential)   (None, 45

In [15]:
m1.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=5, callbacks=[keras.callbacks.ModelCheckpoint('../weights/m1_SCAMPS.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
m1.load_weights('../weights/m1_SCAMPS.h5')

Epoch 1/5
125/125 [==============================] - 3s 17ms/step - loss: 0.8078 - val_loss: 0.7857

Epoch 00001: val_loss improved from inf to 0.78573, saving model to ../weights\m1_SCAMPS.h5
Epoch 2/5
125/125 [==============================] - 2s 16ms/step - loss: 0.7893 - val_loss: 0.7697

Epoch 00002: val_loss improved from 0.78573 to 0.76973, saving model to ../weights\m1_SCAMPS.h5
Epoch 3/5
125/125 [==============================] - 2s 16ms/step - loss: 0.7797 - val_loss: 0.7520

Epoch 00003: val_loss improved from 0.76973 to 0.75200, saving model to ../weights\m1_SCAMPS.h5
Epoch 4/5
125/125 [==============================] - 2s 16ms/step - loss: 0.7410 - val_loss: 0.7324

Epoch 00004: val_loss improved from 0.75200 to 0.73237, saving model to ../weights\m1_SCAMPS.h5
Epoch 5/5
125/125 [==============================] - 2s 16ms/step - loss: 0.6966 - val_loss: 0.6536

Epoch 00005: val_loss improved from 0.73237 to 0.65356, saving model to ../weights\m1_SCAMPS.h5


In [3]:
m1.load_weights('../weights/m1_SCAMPS.h5')

In [16]:
eval_on_dataset(test_set_PURE, m1, 450, (8, 8), step=5, batch=32, save='../results/ours_SCAMPS_PURE.h5')
get_metrics('../results/ours_SCAMPS_PURE.h5',)

100%|██████████| 59/59 [00:01<00:00, 56.93it/s]


{'Sliding window': {'MAE': 2.922, 'RMSE': 11.082, 'R': 0.89064},
 'Whole video': {'MAE': 1.162, 'RMSE': 6.421, 'R': 0.96226}}

In [17]:
eval_on_dataset(test_set_UBFC_rPPG2, m1, 450, (8, 8), step=5, batch=32, save='../results/ours_SCAMPS_UBFC.h5')
get_metrics('../results/ours_SCAMPS_UBFC.h5',)

100%|██████████| 42/42 [00:00<00:00, 58.95it/s]


{'Sliding window': {'MAE': 1.339, 'RMSE': 2.901, 'R': 0.98572},
 'Whole video': {'MAE': 1.35, 'RMSE': 2.61, 'R': 0.98851}}